In [1]:
import cherrypy
import json
import redis
import uuid
import requests
from redis.commands.json.path import Path
import time

In [2]:
publicid = "redis-15072.c77.eu-west-1-1.ec2.cloud.redislabs.com"
username = "default"
password = "53R8YAlL81zAHIEVcPjwjzcnVQoSPhzt"
redis_port = 15072

redis_client = redis.Redis(host = publicid, password = password, username = username, port = redis_port)
print("Is connected? ", redis_client.ping())

Is connected?  True


In [ ]:
class SMS(object):
    exposed = True
    
    def GET(self):
        # response = {'mac_address':[]}
        items = []
        keys = redis_client.keys("*:speed")
        print(len(keys))
        for key in keys:
            key = key.decode()
            items.append(key.split(":")[0])

        response = {"mac_address":items}
        
        return json.dumps(response)


class SpecificSMS:
    exposed = True

    def GET(self, *path, **query):
        mac_address = path[0]
        if mac_address == "":
            raise cherrypy.HTTPError(400, "Bad requested: missing MAC address")
        else:
            items = []
            keys = redis_client.keys("*:speed")
            for key in keys:
                key = key.decode()
                items.append(key.split(":")[0])
            if mac_address not in items:
                raise cherrypy.HTTPError(404, "Not found: invalid MAC address")

        if query['from'] == "":
            raise cherrypy.HTTPError(400, "Bad requested: missing start time")
        if query['to'] == "":
            raise cherrypy.HTTPError(400, "Bad requested: missing end time")

        speeds = redis_client.ts().range(f"{mac_address}:speed", int(query['from']), int(query['to']))
        timestamps = [speed[0] for speed in speeds]
        speed_label = [speed[1] for speed in speeds]

        response = {
            "mac_address":mac_address,
            "timestamps":timestamps,
            "speed_label":speed_label,
        }
        return json.dumps(response)

    def DELETE(self, *path, **query):
        mac_address = path[0]
        if mac_address == "":
            raise cherrypy.HTTPError(400, "Bad requested: missing MAC address")
        else:
            items = []
            keys = redis_client.keys("*:speed")
            for key in keys:
                key = key.decode()
                items.append(key.split(":")[0])
            if mac_address not in items:
                raise cherrypy.HTTPError(404, "Not found: invalid MAC address")

        redis_client.delete(f"{mac_address}:speed")

In [ ]:
if __name__=='__main__':
    conf = {'/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher(), 'tools.sessions.on':True}}
    cherrypy.tree.mount(SMS(), '/devices', conf)
    cherrypy.tree.mount(SpecificSMS(), '/device', conf)
    cherrypy.config.update({'server.socket_host': '0.0.0.0'})
    cherrypy.config.update({'server.socket_port': 8080})
    cherrypy.engine.start()
    cherrypy.engine.block()